In [199]:
import pandas as pd
import numpy as np

In [173]:
df_all = pd.DataFrame({'Id': [], 'TS_comp': [], 'TS_host': [], 'TS_ligand': []})

# cb7_gilson_feprocess

In [174]:
df_com = pd.read_csv('../../Datasets/VM2_data/cb7_gilson_feprocess/cb7_feprocess_complex.csv')
df_lig = pd.read_csv('../../Datasets/VM2_data/cb7_gilson_feprocess/feprocess_ligand.csv')
df_hos = pd.read_csv('../../Datasets/VM2_data/cb7_gilson_feprocess/cb7_feprocess_host.csv')

In [175]:
comp_arr = df_com[' S'].to_numpy()
lig_arr = df_lig[' S'].to_numpy()
hos_arr = df_hos[' S'].to_list()*29
Ids = ['gilson_' + df_com.iloc[i][' Ident'] for i in range(len(comp_arr))]

In [176]:
df_cb7_gilson = pd.DataFrame({'Id': Ids, 'TS_comp': comp_arr, 'TS_host': hos_arr, 'TS_ligand': lig_arr})
df_all = pd.concat([df_all, df_cb7_gilson], ignore_index=True)

# cd_mobley_set2

In [177]:
df_com = pd.read_csv('../../Datasets/VM2_data/cd_mobley_set2/b-cyclodex_feprocess_complex.csv')
df_lig = pd.read_csv('../../Datasets/VM2_data/cd_mobley_set2/feprocess_ligand.csv')
df_hos = pd.read_csv('../../Datasets/VM2_data/cd_mobley_set2/b-cyclodex_feprocess_host.csv')

In [178]:
comp_arr = df_com[' S'].to_numpy()
lig_arr = df_lig[' S'].to_numpy()
hos_arr = df_hos[' S'].to_list()*21
Ids = ['mobley_set2_' + df_com.iloc[i][' Ident'] for i in range(len(comp_arr))]

In [179]:
df_mobley_set2 = pd.DataFrame({'Id': Ids, 'TS_comp': comp_arr, 'TS_host': hos_arr, 'TS_ligand': lig_arr})
df_all = pd.concat([df_all, df_mobley_set2], ignore_index=True)

# Sampl3_h1_and_2_feprocess

In [215]:
df_com = pd.read_csv('../../Datasets/VM2_data/Sampl3_h1_and_2_feprocess/host2_host3_feprocess_complex.csv')
df_lig = pd.read_csv('../../Datasets/VM2_data/Sampl3_h1_and_2_feprocess/feprocess_ligand.csv')
df_hos = pd.read_csv('../../Datasets/VM2_data/Sampl3_h1_and_2_feprocess/host2_host3_feprocess_host.csv')

In [181]:
comp_arr = df_com[' S'].to_numpy()
lig_arr = df_lig[' S'].to_list()*2
hos_arr = [df_hos[' S'].to_list()[0]]*2 + [df_hos[' S'].to_list()[1]]*2
Ids = ['Sampl3_h1_and_2_' + df_com.iloc[i][' Ident'] for i in range(len(comp_arr))]

In [182]:
df_Sampl3_h1_and_2 = pd.DataFrame({'Id': Ids, 'TS_comp': comp_arr, 'TS_host': hos_arr, 'TS_ligand': lig_arr})
df_all = pd.concat([df_all, df_Sampl3_h1_and_2], ignore_index=True)

# Sampl6_OA_feprocess

In [183]:
df_com = pd.read_csv('../../Datasets/VM2_data/Sampl6_OA_feprocess/OA_TEMOA_feprocess_complex.csv')
df_lig = pd.read_csv('../../Datasets/VM2_data/Sampl6_OA_feprocess/feprocess_ligand.csv')
df_hos = pd.read_csv('../../Datasets/VM2_data/Sampl6_OA_feprocess/OA_TEMOA_feprocess_host.csv')

In [184]:
comp_arr = df_com[' S'].to_numpy()
lig_arr = df_lig[' S'].to_list()*2
hos_arr = [df_hos[' S'].to_list()[0]]*8 + [df_hos[' S'].to_list()[1]]*8
Ids = ['Sampl6_OA_' + df_com.iloc[i][' Ident'] for i in range(len(comp_arr))]

In [185]:
df_Sampl6_OA_feprocess = pd.DataFrame({'Id': Ids, 'TS_comp': comp_arr, 'TS_host': hos_arr, 'TS_ligand': lig_arr})
df_all = pd.concat([df_all, df_Sampl6_OA_feprocess], ignore_index=True)

# Sampl6_cb8_feprocess

In [186]:
df_com = pd.read_csv('../../Datasets/VM2_data/Sampl6_cb8_feprocess/CB8_feprocess_complex.csv')
df_lig = pd.read_csv('../../Datasets/VM2_data/Sampl6_cb8_feprocess/feprocess_ligand.csv')
df_hos = pd.read_csv('../../Datasets/VM2_data/Sampl6_cb8_feprocess/CB8_feprocess_host.csv')

In [187]:
comp_arr = df_com[' S'].to_numpy()
lig_arr = df_lig[' S'].to_numpy()
hos_arr = df_hos[' S'].to_list()*11
Ids = ['Sampl6_cb8_' + df_com.iloc[i][' Ident'] for i in range(len(comp_arr))]

In [188]:
df_Sampl6_cb8_feprocess = pd.DataFrame({'Id': Ids, 'TS_comp': comp_arr, 'TS_host': hos_arr, 'TS_ligand': lig_arr})
df_all = pd.concat([df_all, df_Sampl6_cb8_feprocess], ignore_index=True)

In [217]:
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest1')[0][0], ['exp']] = -0.9
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest2')[0][0], ['exp']] = -0.4
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest3')[0][0], ['exp']] = -0.1
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest4')[0][0], ['exp']] = 0.6
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest5')[0][0], ['exp']] = 0.0
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest6')[0][0], ['exp']] = -0.1
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest7')[0][0], ['exp']] = -1.2
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest8')[0][0], ['exp']] = -0.7
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest9')[0][0], ['exp']] = 0.4
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest10')[0][0], ['exp']] = -0.5
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest11')[0][0], ['exp']] = -0.7
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest12')[0][0], ['exp']] = -1.7
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest13')[0][0], ['exp']] = 0.4
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest14')[0][0], ['exp']] = -0.3
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest15')[0][0], ['exp']] = -0.3
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest16')[0][0], ['exp']] = 0.9
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest17')[0][0], ['exp']] = -0.8
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest18')[0][0], ['exp']] = -0.4
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest19')[0][0], ['exp']] = -0.5
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest19')[0][0], ['exp']] = -0.5
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest20')[0][0], ['exp']] = 0.4
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest21')[0][0], ['exp']] = -0.6
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest22')[0][0], ['exp']] = -0.1
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest23')[0][0], ['exp']] = -0.8
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest24')[0][0], ['exp']] = -0.8
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest25')[0][0], ['exp']] = 0.0
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest26')[0][0], ['exp']] = -0.2
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest27')[0][0], ['exp']] = -0.6
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest28')[0][0], ['exp']] = 0.2
df_all.loc[np.where(df_all['Id'] == 'gilson_cb7--guest29')[0][0], ['exp']] = -0.2
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-1')[0][0], ['exp']] = -3.086
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-2')[0][0], ['exp']] = -3.105
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-3')[0][0], ['exp']] = -1.96
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-4')[0][0], ['exp']] = -2.371
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-5')[0][0], ['exp']] = -1.6
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-6')[0][0], ['exp']] = -3.157
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-7')[0][0], ['exp']] = -3.814
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-8')[0][0], ['exp']] = -0.87
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-9')[0][0], ['exp']] = 0.093
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-10')[0][0], ['exp']] = 0.698
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-11')[0][0], ['exp']] = -0.552
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-12')[0][0], ['exp']] = -0.779
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s13')[0][0], ['exp']] = -3.027
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s14')[0][0], ['exp']] = -2.434
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s15')[0][0], ['exp']] = -1.885
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s16')[0][0], ['exp']] = -2.049
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s17')[0][0], ['exp']] = -1.051
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s18')[0][0], ['exp']] = -3.59
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s19')[0][0], ['exp']] = 1.292
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s20')[0][0], ['exp']] = 0.774
df_all.loc[np.where(df_all['Id'] == 'mobley_set2_b-cyclodex--guest-s21')[0][0], ['exp']] = -1.268
df_all.loc[np.where(df_all['Id'] == 'Sampl3_h1_and_2_host2--guest8')[0][0], ['exp']] = 2.38
df_all.loc[np.where(df_all['Id'] == 'Sampl3_h1_and_2_host2--guest9')[0][0], ['exp']] = -3.63
df_all.loc[np.where(df_all['Id'] == 'Sampl3_h1_and_2_host3--guest8')[0][0], ['exp']] = -2.4
df_all.loc[np.where(df_all['Id'] == 'Sampl3_h1_and_2_host3--guest9')[0][0], ['exp']] = -4.79
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G0')[0][0], ['exp']] = -0.8
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G1')[0][0], ['exp']] = 0.9
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G2')[0][0], ['exp']] = 3.7
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G3')[0][0], ['exp']] = 2.4
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G4')[0][0], ['exp']] = -0.2
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G5')[0][0], ['exp']] = 0.7
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G6')[0][0], ['exp']] = 0.3
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_OA--OA-G7')[0][0], ['exp']] = 1.2
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G0')[0][0], ['exp']] = 1.8
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G1')[0][0], ['exp']] = 2.3
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G2')[0][0], ['exp']] = 2.5
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G3')[0][0], ['exp']] = 3.2
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G4')[0][0], ['exp']] = 1.1
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G5')[0][0], ['exp']] = 3.8
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G6')[0][0], ['exp']] = 0.8
df_all.loc[np.where(df_all['Id'] == 'Sampl6_OA_TEMOA--OA-G7')[0][0], ['exp']] = 4.2
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G0')[0][0], ['exp']] = -2.5
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G1')[0][0], ['exp']] = -2.6
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G2')[0][0], ['exp']] = -1.2
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G3')[0][0], ['exp']] = -4.0
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G4')[0][0], ['exp']] = 2.0
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G5')[0][0], ['exp']] = -5.0
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G6')[0][0], ['exp']] = -2.6
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G7')[0][0], ['exp']] = -3.5
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G8')[0][0], ['exp']] = 0.9
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G9')[0][0], ['exp']] = -4.0
df_all.loc[np.where(df_all['Id'] == 'Sampl6_cb8_CB8--CB8-G10')[0][0], ['exp']] = -5.7

In [212]:
df_all = df_all.dropna()

In [214]:
df_all[df_all['exp'] == None]

,Id,TS_comp,TS_host,TS_ligand,exp
